# Chapter FIVE: Combining LangChain and Llamalndex into ROBOAGENT

In [5]:
!pip install  llama-index-readers-wikipedia

In [7]:
# Import necessary modules
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from llama_index.vector_stores import PineconeVectorStore
from llama_index.core import VectorStoreIndex
from pinecone import Pinecone, ServerlessSpec
import os

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Define the LCEL Runnable
llm = ChatOpenAI(temperature=0)
prompt_template_question = "What is the answer to this financial question: {question}?"
runnable = PromptTemplate.from_template(prompt_template_question) | llm

# Define the Pinecone Serverless as the data source
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("mrag-fin-docs")
vector_store = PineconeVectorStore(pinecone_index=index)
# Define the LlamaIndex
llama_index = VectorStoreIndex.From_Vector_Store(index)
# Combine the LCEL Runnable and LlamaIndex
class FinancialAnalystAgent:
    def __init__(self, runnable, llama_index):
        self.runnable = runnable
        self.llama_index = llama_index

    def query(self, question):
        # Use the runnable to generate an answer
        answer = self.runnable.invoke({"question": question})
        
        # Use the llama_index to retrieve relevant documents
        relevant_documents = self.llama_index.as_query_engine().query(answer)
        
        return answer, relevant_documents

# Instantiate the agent
agent = FinancialAnalystAgent(runnable, llama_index)

# Use the agent
question = "What is the total revenue for the last fiscal year?"
answer, relevant_documents = agent.query(question)
print(f"Answer: {answer}")
print(f"Relevant Documents: {relevant_documents}")


ImportError: cannot import name 'PineconeVectorStore' from 'llama_index.vector_stores' (unknown location)